In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Morris.sql") \
      .getOrCreate() 

In [5]:
# Create RDD from parallelize    
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)

In [6]:
type(rdd)

pyspark.rdd.RDD

In [7]:
# 1. Load the data
df_capstone = spark.read.csv("Capstone market analysis (4).csv",header = True)
df_capstone.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [9]:
# Create the temporary view of dataset to query in SQL
df_capstone.createOrReplaceTempView("df_capst")

df2 = spark.sql("select * from df_capst")

df2.printSchema()

df2.show()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+----

In [10]:
# TOTAL COUNT
df3 = spark.sql("select count(*) from df_capst")

df3.show()

+--------+
|count(1)|
+--------+
|   45211|
+--------+



In [8]:
# Status of marital
df4 = spark.sql("select marital , count(*) as Cnt from df_capst group by marital ")

df4.show()

+--------+-----+
| marital|  Cnt|
+--------+-----+
|divorced| 5207|
| married|27214|
|  single|12790|
+--------+-----+



In [13]:
df5 = spark.sql("select job,count(*) as cnt from df_capst group by job ")
df5.createOrReplaceTempView("temp2")

df5.show()

+-------------+----+
|          job| cnt|
+-------------+----+
|   management|9458|
|      retired|2264|
|      unknown| 288|
|self-employed|1579|
|      student| 938|
|  blue-collar|9732|
| entrepreneur|1487|
|       admin.|5171|
|   technician|7597|
|     services|4154|
|    housemaid|1240|
|   unemployed|1303|
+-------------+----+



In [14]:
# Is the divorece has something to do with job profile 

df7 = spark.sql("select * from df_capst where marital = 'divorced' ")
#df7.show()

df7.createOrReplaceTempView("temp")

df6 = spark.sql("select job , count(*) as d_cnt from temp group by job ")
df6.createOrReplaceTempView("temp1")

In [15]:
df8 = spark.sql("select a.*,b.d_cnt ,d_cnt/cnt as per from temp2 a left join temp1 b  on a.job = b.job")

df8.show()

+-------------+----+-----+--------------------+
|          job| cnt|d_cnt|                 per|
+-------------+----+-----+--------------------+
|   management|9458| 1111| 0.11746669486149292|
|      retired|2264|  425|  0.1877208480565371|
|      unknown| 288|   17|0.059027777777777776|
|self-employed|1579|  140| 0.08866371120962635|
|      student| 938|    6|0.006396588486140725|
|  blue-collar|9732|  750| 0.07706535141800247|
| entrepreneur|1487|  179| 0.12037659717552118|
|       admin.|5171|  750| 0.14503964416940632|
|   technician|7597|  925| 0.12175858891667764|
|     services|4154|  549| 0.13216177178623015|
|    housemaid|1240|  184| 0.14838709677419354|
|   unemployed|1303|  171| 0.13123561013046814|
+-------------+----+-----+--------------------+



In [16]:
df_capstone.createOrReplaceTempView("sample")
df2 = spark.sql("SELECT * from sample ")
df2.printSchema()
df2.show()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+----

In [19]:
df2 = df2.withColumnRenamed("Sex","Gender") 
df2.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [ ]:
# 1. Change the column name (age -- age_values)
# Change the data type (age_value , string --- integer)

In [22]:
df2_capstone = df_capstone.withColumnRenamed("age","age_values")

In [24]:
df2_capstone.printSchema()

root
 |-- age_values: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [25]:
df2_capstone.show()

+----------+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age_values|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+----------+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|        58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
|        44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
|        33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
|        47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
|        33|     unknown|  single|

In [28]:
temp = df2_capstone.withColumn("age_values_1",df2_capstone.age_values * 10)

temp.printSchema()


#df.withColumn("salary",col("salary")*100).show()


root
 |-- age_values: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)
 |-- age_values_1: double (nullable = true)



In [29]:
temp.show()

+----------+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+------------+
|age_values|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|age_values_1|
+----------+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+------------+
|        58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|       580.0|
|        44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|       440.0|
|        33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|       330.0|
|        47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|   

In [30]:
df_capstone.drop('age_values_1').show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [31]:
df2_capstone.filter((df2_capstone.marital == 'married') & (df2_capstone.job == 'management')).count()

5400

In [32]:
a = ['married','single']

df2_capstone.filter(df2_capstone.marital.isin(a)).show() 

+----------+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age_values|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+----------+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|        58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
|        44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
|        33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
|        47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
|        33|     unknown| single|  unknow

In [36]:
df2_capstone = df2_capstone.withColumnRenamed("dob","DateOfBirth") \
    .withColumnRenamed("salary","salary_amount")
df2_capstone.printSchema()


df2_capstone.withColumn("salary",col=("salary").cast("Integer")).show()


root
 |-- age_values: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



AttributeError: 'str' object has no attribute 'cast'

In [ ]:
1. Change DataType using PySpark withColumn()